<a href="https://colab.research.google.com/github/WajdAlsuhaymi/IT326-DataMining-Group2/blob/main/Reports/Phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 1 – Data Selection

## Project Goal
The main goal of this project is to explore the relationship between social media engagement and emotional well-being. Through data analysis, the project aims to uncover how factors such as time spent, purpose of use, and users’ demographic characteristics influence psychological outcomes. The project seeks to raise awareness about the psychological effects of social media.

## Dataset Source
Kaggle: [Social Media Usage and Emotional Well-being](https://www.kaggle.com/datasets/emirhanai/social-media-usage-and-emotional-well-being)


### Read the file

In [18]:
import pandas as pd


url = "https://raw.githubusercontent.com/WajdAlSuhaymi/IT326-DataMining-Group2/main/Dataset/Raw_dataset.csv"

# قراءة الملف
df = pd.read_csv(url)

# عرض أول 5 صفوف للتأكد
df.head()


,User_ID,Age,Gender,Platform,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day,Dominant_Emotion
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,25,Female,Instagram,120.0,3.0,45.0,10.0,12.0,Happiness
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,30,Male,Twitter,90.0,5.0,20.0,25.0,30.0,Anger
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dataset Description and **Exploration**

In [19]:
import pandas as pd



# --------------------------------------------
# معلومات عامة عن البيانات
print("\n🔹 General Info:")
df.info()

# --------------------------------------------
# عدد السجلات (الصفوف) وعدد الأعمدة
print("\n🔹 Dataset shape (rows, columns):", df.shape)

# --------------------------------------------
# التحقق من القيم المفقودة (اختياري لكنه مفيد)
print("\n🔹 Missing values per column:")
print(df.isnull().sum())

# --------------------------------------------
# تحديد العمود الخاص بالتصنيف (Class Attribute)
# في هذه البيانات العمود هو 'Emotion' أو 'Dominant_Emotion' حسب الاسم في ملفك
class_column = 'Dominant_Emotion' # غيّري الاسم إذا مكتوب 'Dominant_Emotion'

# القيم (الفئات) الموجودة في هذا العمود
print("\n🔹 Unique emotion labels:")
print(df[class_column].unique())

# --------------------------------------------
# عدد السجلات في كل فئة (label)
print("\n🔹 Count of instances per label:")
print(df[class_column].value_counts())

# --------------------------------------------
# أنواع البيانات (Attributes types)
print("\n🔹 Data types of attributes:")
print(df.dtypes)

# --------------------------------------------


🔹 General Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2004 entries, 0 to 2003
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     1001 non-null   object 
 1   Age                         1001 non-null   object 
 2   Gender                      1000 non-null   object 
 3   Platform                    1000 non-null   object 
 4   Daily_Usage_Time (minutes)  1000 non-null   float64
 5   Posts_Per_Day               1000 non-null   float64
 6   Likes_Received_Per_Day      1000 non-null   float64
 7   Comments_Received_Per_Day   1000 non-null   float64
 8   Messages_Sent_Per_Day       1000 non-null   float64
 9   Dominant_Emotion            1000 non-null   object 
dtypes: float64(5), object(5)
memory usage: 156.7+ KB

🔹 Dataset shape (rows, columns): (2004, 10)

🔹 Missing values per column:
User_ID                       1003
Age                         

### Normalize Gender Attribute

In [20]:
import re
import numpy as np
import pandas as pd

# --- تنظيف وضبط عمود Gender بحيث يكون فقط: Male / Female / Non-binary ---
def normalize_gender(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip().str.lower()

    # خرائط سريعة للقيم الشائعة
    mapping = {
        "m":"male","male":"male","man":"male","boy":"male","masculine":"male","1":"male",
        "f":"female","female":"female","woman":"female","girl":"female","feminine":"female","0":"female",
        "nb":"non-binary","nonbinary":"non-binary","non-binary":"non-binary","non binary":"non-binary",
        "2":"non-binary","other":"non-binary","others":"non-binary"
    }

    # تحويل مباشر إن وُجدت مطابقة كاملة
    s = s.map(lambda x: mapping.get(x, x))

    # محاولات regex للأخطاء الإملائية المحتملة
    s = s.str.replace(r'[^a-z\- ]', '', regex=True)          # إزالة رموز/أرقام
    s = s.str.replace(r'\s+', ' ', regex=True).str.strip()

    # تصنيف بالكلمات المفتاحية
    s = s.replace({
        r'.*\bmale\b.*': 'male',
        r'.*\bfemale\b.*': 'female',
        r'.*\bnon[- ]?binary\b.*': 'non-binary',
        r'^\b(m|man|boy)\b$': 'male',
        r'^\b(f|woman|girl)\b$': 'female',
        r'^\b(nb)\b$': 'non-binary',
    }, regex=True)

    # أي شيء ليس إحدى القيم الثلاث → NaN (نلغيّه بدل ما يطلع أرقام)
    s = s.where(s.isin(["male","female","non-binary"]), np.nan)

    # أرجعها بحالة عنوانية
    return s.map({"male":"Male","female":"Female","non-binary":"Non-binary"})

# طبّقي على داتا سِتك
df["Gender"] = normalize_gender(df["Gender"])

# --- تلخيص Gender فقط بشكل ثابت بالترتيب المطلوب ---
present = [v for v in ["Male","Female","Non-binary"] if v in df["Gender"].dropna().unique().tolist()]
gender_summary = " - ".join(present) if present else "—"
print("Gender → Nominal:", gender_summary)


Gender → Nominal: Male - Female - Non-binary


### Summarize Attributes

In [23]:
import pandas as pd, numpy as np, re

# =============== 0) قراءة الداتا ===============
try:
    df  # إن كانت df موجودة مسبقًا
except NameError:
    df = pd.read_csv("Raw_dataset.csv")

# =============== 1) تنظيف Gender ثم تثبيته ===============
def normalize_gender(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip().str.lower()

    mapping = {
        "m":"male","male":"male","man":"male","boy":"male","masculine":"male","1":"male",
        "f":"female","female":"female","woman":"female","girl":"female","feminine":"female","0":"female",
        "nb":"non-binary","nonbinary":"non-binary","non-binary":"non-binary","non binary":"non-binary",
        "2":"non-binary","other":"non-binary","others":"non-binary"
    }
    s = s.map(lambda x: mapping.get(x, x))
    s = s.str.replace(r'[^a-z\- ]', '', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()
    s = s.replace({
        r'.*\bmale\b.*': 'male',
        r'.*\bfemale\b.*': 'female',
        r'.*\bnon[- ]?binary\b.*': 'non-binary',
        r'^\b(m|man|boy)\b$': 'male',
        r'^\b(f|woman|girl)\b$': 'female',
        r'^\b(nb)\b$': 'non-binary',
    }, regex=True)
    s = s.where(s.isin(["male","female","non-binary"]), np.nan)
    return s.map({"male":"Male","female":"Female","non-binary":"Non-binary"})

if "Gender" in df.columns:
    df["Gender"] = normalize_gender(df["Gender"])

# =============== 2) أدوات مساعدة للتلخيص ===============
def to_numeric_or_range(series: pd.Series):
    s = series.astype(str).str.strip()
    out = pd.Series(np.nan, index=s.index)
    for i, val in s.items():
        if re.fullmatch(r"\d+(\.\d+)?", val):
            out.at[i] = float(val)
        elif re.fullmatch(r"\d+\s*-\s*\d+", val):
            a, b = map(float, re.findall(r"\d+", val))
            out.at[i] = (a + b) / 2
    return out

def summarize_one(col: str, max_list=15):
    s = df[col].dropna()

    # --- الشرط الخاص بـ Gender: دائماً Nominal بالقيم الثلاث ---
    if col == "Gender":
        return "Nominal", "Male - Female - Non-binary"

    # محاولة اعتباره رقميًا مباشرة
    num = pd.to_numeric(s, errors="coerce")
    if num.notna().any():
        mn, mx = num.min(), num.max()
        fmt = lambda v: f"{int(v)}" if float(v).is_integer() else f"{v:.4g}"
        return "Numeric", f"{fmt(mn)} - {fmt(mx)}"

    # محاولة تحويل نطاقات نصية لأرقام
    num2 = to_numeric_or_range(s)
    if num2.notna().any():
        mn, mx = num2.min(), num2.max()
        fmt = lambda v: f"{int(v)}" if float(v).is_integer() else f"{v:.4g}"
        return "Numeric", f"{fmt(mn)} - {fmt(mx)}"

    # اسمي (Nominal)
    uniq = sorted(s.astype(str).str.strip().unique(), key=lambda x: x.lower())
    vals = " - ".join(uniq[:max_list]) + (f" - ... (+{len(uniq)-max_list} more)" if len(uniq) > max_list else "")
    return "Nominal", (vals if vals else "—")

# =============== 3) بناء جدول الملخص ===============
order = [
    "User_ID","Age","Gender","Platform",
    "Daily_Usage_Time (minutes)","Posts_Per_Day","Likes_Received_Per_Day",
    "Comments_Received_Per_Day","Messages_Sent_Per_Day","Dominant_Emotion"
]

rows = []
for col in order:
    if col not in df.columns:
        rows.append({"Attribute": f"{col} (NOT FOUND)", "Type": "-", "Range / Possible Values": "-"})
        continue
    t, rng = summarize_one(col, max_list=15)
    rows.append({"Attribute": col, "Type": t, "Range / Possible Values": rng})

summary = pd.DataFrame(rows)
summary["__o__"] = summary["Type"].map({"Numeric": 0, "Nominal": 1}).fillna(2)
summary = summary.sort_values(["__o__", "Attribute"]).drop(columns="__o__").reset_index(drop=True)

# عرض النتيجة
summary


,Attribute,Type,Range / Possible Values
0,Age,Numeric,21 - 35
1,Comments_Received_Per_Day,Numeric,2 - 40
2,Daily_Usage_Time (minutes),Numeric,40 - 200
3,Likes_Received_Per_Day,Numeric,5 - 110
4,Messages_Sent_Per_Day,Numeric,8 - 50
5,Posts_Per_Day,Numeric,1 - 8
6,User_ID,Numeric,1 - 1000
7,Dominant_Emotion,Nominal,Anger - Anxiety - Boredom - Happiness - Neutra...
8,Gender,Nominal,Male - Female - Non-binary
9,Platform,Nominal,Facebook - Instagram - LinkedIn - Snapchat - T...
